In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
file = 'D:/data/wedling gun/E016 2021 18.csv'#10th5028096-1003_2035有缺失值，10th5495048-1004_3598有离群点
df = pd.read_csv(file, index_col='time',low_memory=False)
df.index = pd.to_datetime(df.index).tz_localize(None)
df = df.resample('S').asfreq()
df=df.apply(pd.to_numeric, errors='ignore')
df[df['in_Sheet_thickness']>5] = np.nan
df[df['in_Sheet_thickness']<=0] = np.nan

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder, MinMaxScaler
list = [2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19]
df.iloc[:, list]= StandardScaler().fit_transform(df.iloc[:, list])# 数值类型特征标准化
L = df.iloc[:, list]
L_imputed=df.iloc[:, list]
L_imputed.fillna(method='ffill', inplace=True)
L_imputed

In [ ]:
LEN=1200
AHEAD=60
HIS=120
LIST_TAR=[2]
LIST_COVS=[1,3,4,9,10,11,12,13,14,16]

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.datasets import AirPassengersDataset
from darts.metrics import coefficient_of_variation,mae,mape,marre,mase,mse
#data_tar = L_imputed.iloc[-LEN:, LIST_TAR]
#data_covs = L_imputed.iloc[-LEN:, LIST_COVS]
data_tar = L_imputed.iloc[-LEN-650:-650, LIST_TAR]
data_covs = L_imputed.iloc[-LEN-650:-650, LIST_COVS]
series_tar = TimeSeries.from_dataframe(data_tar,freq='S').astype(np.float32)
train, test = series_tar[:-AHEAD], series_tar[-AHEAD:]
series_covs = TimeSeries.from_dataframe(data_covs,freq='S').astype(np.float32)
past_covs,future_cosv = series_covs[:-AHEAD], series_covs[-AHEAD:]

In [ ]:
from darts.models import LightGBMModel

model_LightGBMModel = LightGBMModel(
    lags=HIS, 
    lags_past_covariates=HIS,
    output_chunk_length=AHEAD, 
    random_state =43, 
    
)
model_LightGBMModel.fit(train,past_covariates=past_covs)
predl_LightGBMModel = model_LightGBMModel.predict(series=train,n=AHEAD,past_covariates=series_covs)

# # scale back:
# pred = scaler.inverse_transform(pred)

plt.figure(figsize=(35, 10))
train[-150:].plot(label="train")
test.plot(label="actual")
predl_LightGBMModel.plot(label="forecast")
print("_LightGBMModel coefficient_of_variation:", coefficient_of_variation(test, predl_LightGBMModel))
print("_LightGBMModel mae:", mae(test, predl_LightGBMModel))
print("_LightGBMModel mape:", mape(test, predl_LightGBMModel))
print("_LightGBMModel marre:", marre(test, predl_LightGBMModel))
print("_LightGBMModel mse:", mse(test, predl_LightGBMModel))